In [ ]:
import zipfile
import os

# Paths to your zip files
zip_paths = [
    "/content/augmented_images.zip",
    "/content/augmented_images(white throated kingfisher).zip",
    "/content/augmented_images(red vented bulbul).zip"
]

# Unzipping function
def unzip_files(zip_paths, extract_to_base):
    for zip_path in zip_paths:
        # Get the folder name based on the zip file name
        folder_name = os.path.splitext(os.path.basename(zip_path))[0]
        extract_to = os.path.join(extract_to_base, folder_name)

        os.makedirs(extract_to, exist_ok=True)
        print(f"Unzipping {zip_path} to {extract_to}...")

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Finished unzipping {zip_path} to {extract_to}")

# Unzip all files to separate directories for each bird
extracted_base_dir = "/content/extracted_images"
unzip_files(zip_paths, extracted_base_dir)


Unzipping /content/augmented_images.zip to /content/extracted_images/augmented_images...
Finished unzipping /content/augmented_images.zip to /content/extracted_images/augmented_images
Unzipping /content/augmented_images(white throated kingfisher).zip to /content/extracted_images/augmented_images(white throated kingfisher)...
Finished unzipping /content/augmented_images(white throated kingfisher).zip to /content/extracted_images/augmented_images(white throated kingfisher)
Unzipping /content/augmented_images(red vented bulbul).zip to /content/extracted_images/augmented_images(red vented bulbul)...
Finished unzipping /content/augmented_images(red vented bulbul).zip to /content/extracted_images/augmented_images(red vented bulbul)


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Function to split and organize datasets
def split_and_organize_datasets(base_extracted_dir, split_base_dir, test_size=0.2, val_size=0.2):
    # Clear or create split directories
    if os.path.exists(split_base_dir):
        shutil.rmtree(split_base_dir)
    os.makedirs(split_base_dir, exist_ok=True)

    # Create subdirectories for train, validation, and test
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(split_base_dir, split), exist_ok=True)

    # Loop through each bird folder and split its images
    for bird_folder in os.listdir(base_extracted_dir):
        bird_dir = os.path.join(base_extracted_dir, bird_folder)
        if not os.path.isdir(bird_dir):
            continue

        print(f"Processing {bird_folder}...")

        # Get all image files in the current bird folder
        images = os.listdir(bird_dir)
        images = [img for img in images if img.endswith(('.jpg', '.jpeg', '.png'))]  # Filter valid image files

        if len(images) == 0:
            print(f"No valid image files found in {bird_folder}")
            continue

        # Split the images into train, test, and validation sets
        train_imgs, test_imgs = train_test_split(images, test_size=test_size, random_state=42)
        train_imgs, val_imgs = train_test_split(train_imgs, test_size=val_size, random_state=42)

        # Copy images to the corresponding directories
        for split, imgs in zip(['train', 'val', 'test'], [train_imgs, val_imgs, test_imgs]):
            split_dir = os.path.join(split_base_dir, split, bird_folder)
            os.makedirs(split_dir, exist_ok=True)
            for img in imgs:
                shutil.copy(os.path.join(bird_dir, img), os.path.join(split_dir, img))

# Path to your extracted dataset directory
base_extracted_dir = "/content/extracted_images"
split_base_dir = "/content/split_datasets"
split_and_organize_datasets(base_extracted_dir, split_base_dir)


Processing augmented_images(red vented bulbul)...
Processing augmented_images(white throated kingfisher)...
Processing augmented_images...


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Image size and batch size settings
img_size = (224, 224)
batch_size = 32

# Create the ImageDataGenerator objects for train, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Define the directories for train, validation, and test
train_dir = os.path.join("/content/split_datasets", 'train')
val_dir = os.path.join("/content/split_datasets", 'val')
test_dir = os.path.join("/content/split_datasets", 'test')

# Create data generators for train, validation, and test
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 1920 images belonging to 3 classes.
Found 480 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load the MobileNetV2 model with pre-trained weights
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the base model layers
base_model.trainable = False

# Create the full model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(train_gen.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])




In [ ]:
# Train the model
epochs = 10
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    verbose=1
)


Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.7188 - loss: 0.6682 - val_accuracy: 0.9563 - val_loss: 0.1967
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - accuracy: 0.9642 - loss: 0.1480 - val_accuracy: 0.9646 - val_loss: 0.1299
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - accuracy: 0.9685 - loss: 0.1141 - val_accuracy: 0.9729 - val_loss: 0.1026
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.9864 - loss: 0.0681 - val_accuracy: 0.9792 - val_loss: 0.0879
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 134s 2s/step - accuracy: 0.9920 - loss: 0.0551 - val_accuracy: 0.9792 - val_loss: 0.0819
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.9949 - loss: 0.0410 - val_accuracy: 0.9854 - val_loss: 0.0698
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 117s 2s/step - accuracy: 0.9957 - loss: 0.0359 - val_accuracy: 0.9875 - val_loss: 0.0684
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 161s 2s/step - accuracy: 0.9973 - loss: 0.0319 - val_accuracy: 0.9896 - v

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# Calculate the average training accuracy across all epochs
average_training_accuracy = sum(history.history['accuracy']) / len(history.history['accuracy'])

# Print the result
print(f"Average Training Accuracy: {average_training_accuracy:.4f}")

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(test_gen, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Generate classification report
y_true = test_gen.classes  # Ground truth labels
y_pred = np.argmax(model.predict(test_gen), axis=-1)  # Predicted labels
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=test_gen.class_indices.keys()))


Average Training Accuracy: 0.9740
19/19 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.9865 - loss: 0.0360
Test Accuracy: 98.33%
19/19 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step
Classification Report:
                                             precision    recall  f1-score   support

                           augmented_images       0.99      0.99      0.99       200
        augmented_images(red vented bulbul)       0.99      0.97      0.98       200
augmented_images(white throated kingfisher)       0.97      0.99      0.98       200

                                   accuracy                           0.98       600
                                  macro avg       0.98      0.98      0.98       600
                               weighted avg       0.98      0.98      0.98       600

